In [1]:
import pymatgen as mg
from mendeleev import element
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("descriptor_ABX2.csv")
df1=df['compound']
df1.head()

0    ZnGeN2  
1    ZnGeN2  
2    ZnGeN2  
3    AlGaN2  
4    AlGaN2  
Name: compound, dtype: object

In [8]:
df.head()

compound     AM1  AN1  Period1  group1  First_IE1  Second_IE1  elctron_AF1  \
0  ZnGeN2    65.390   30        4      12      906.4      1733.3          0.0   
1  ZnGeN2    65.390   30        4      12      906.4      1733.3          0.0   
2  ZnGeN2    65.390   30        4      12      906.4      1733.3          0.0   
3  AlGaN2    26.981   13        3      13      577.5      1816.7        -41.8   
4  AlGaN2    26.981   13        3      13      577.5      1816.7        -41.8   

   paulielctrneg1    Allen_electneg1  ...  heat_fus3  heat_vap3  thermal_con3  \
0              1.65            1.590  ...       0.36       2.79       0.00026   
1              1.65            1.590  ...       0.36       2.79       0.00026   
2              1.65            1.590  ...       0.36       2.79       0.00026   
3              1.61            1.613  ...       0.36       2.79       0.00026   
4              1.61            1.613  ...       0.36       2.79       0.00026   

   spec_heat3  ST  KT  CP  BandGap  Ksgap   Volume  
0      1040.0   1   0   0     2.44   1.76  181.389  
1      1040.0   0   1   0     3.02   2.15  181.833  
2      1040.0   0   0   1     2.73   1.95  181.565  
3      1040.0   1   0   0     4.59   3.28  178.460  
4      1040.0   0   1   0     4.63   3.31  178.365  

[5 rows x 67 columns]

### BY using StrToComposition we seperate compound name into element name   

In [9]:
from matminer.featurizers.conversions import StrToComposition
df = StrToComposition().featurize_dataframe(df, "compound")


#### In new data frame last column have composition name 

In [10]:
df.head()

compound     AM1  AN1  Period1  group1  First_IE1  Second_IE1  elctron_AF1  \
0  ZnGeN2    65.390   30        4      12      906.4      1733.3          0.0   
1  ZnGeN2    65.390   30        4      12      906.4      1733.3          0.0   
2  ZnGeN2    65.390   30        4      12      906.4      1733.3          0.0   
3  AlGaN2    26.981   13        3      13      577.5      1816.7        -41.8   
4  AlGaN2    26.981   13        3      13      577.5      1816.7        -41.8   

   paulielctrneg1    Allen_electneg1  ...  heat_vap3  thermal_con3  \
0              1.65            1.590  ...       2.79       0.00026   
1              1.65            1.590  ...       2.79       0.00026   
2              1.65            1.590  ...       2.79       0.00026   
3              1.61            1.613  ...       2.79       0.00026   
4              1.61            1.613  ...       2.79       0.00026   

   spec_heat3  ST  KT  CP  BandGap  Ksgap   Volume  composition  
0      1040.0   1   0   0     2.44   1.76  181.389  (Zn, Ge, N)  
1      1040.0   0   1   0     3.02   2.15  181.833  (Zn, Ge, N)  
2      1040.0   0   0   1     2.73   1.95  181.565  (Zn, Ge, N)  
3      1040.0   1   0   0     4.59   3.28  178.460  (Al, Ga, N)  
4      1040.0   0   1   0     4.63   3.31  178.365  (Al, Ga, N)  

[5 rows x 68 columns]

In [16]:
df[['compound','composition']].head()

compound  composition
0  ZnGeN2    (Zn, Ge, N)
1  ZnGeN2    (Zn, Ge, N)
2  ZnGeN2    (Zn, Ge, N)
3  AlGaN2    (Al, Ga, N)
4  AlGaN2    (Al, Ga, N)

#### Now we are manualy changing  csv file of composition name i.e. remove bracket and comma

In [14]:
df2=df['composition'] 
df2.to_csv('element.csv')
pd.read_csv('element.csv').head()

/home/enamullah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



0  Zn1 Ge1 N2
0  1  Zn1 Ge1 N2
1  2  Zn1 Ge1 N2
2  3  Al1 Ga1 N2
3  4  Al1 Ga1 N2
4  5  Al1 Ga1 N2

#### Now we are reading manualy changed  csv file of composition name

In [13]:
df3=pd.read_csv("element1.csv")
df3.head()

A   B  X
0  Zn  Ge  N
1  Zn  Ge  N
2  Zn  Ge  N
3  Al  Ga  N
4  Al  Ga  N

In [15]:
df3.shape

(99, 3)

In [18]:
df.reset_index(drop=True, inplace=True)
df3.reset_index(drop=True, inplace=True)
df_U = pd.concat([df3, df1], axis=1)
df_U.head()

A   B  X  compound
0  Zn  Ge  N  ZnGeN2  
1  Zn  Ge  N  ZnGeN2  
2  Zn  Ge  N  ZnGeN2  
3  Al  Ga  N  AlGaN2  
4  Al  Ga  N  AlGaN2

In [35]:
new_df = pd.DataFrame()
for item in df_U['A']:
    comp = mg.Composition(item)
    elements = comp.chemical_system.split("-")
    properties = []
    for i in elements:
            ele = element(i)  
            AN = ele.atomic_number
            AM = ele.mass
            P  = ele.period
            G = ele.group_id    
            IE = ele.ionenergies           
            IE_main1 = IE[1] 
            IE_main2 = IE[2]            
            EAf= ele.electron_affinity
            PEneg = ele.en_pauling
            AEneg = ele.en_allen
            Vand_R = ele.vdw_radius
            Cov_R = ele.covalent_radius
            Atom_R = ele.atomic_radius
            Melting_P = ele.melting_point
            Boiling_P = ele.boiling_point
            density = ele.density
            HeatOfFusion = ele.fusion_heat 
            HeatOfEvop = ele.evaporation_heat
            ThermalCound = ele.thermal_conductivity
            SpecficHeat = ele.specific_heat
            properties.append([AN, AM, P, G, IE_main1, IE_main2, EAf, PEneg, AEneg,Vand_R, Cov_R, Atom_R, Melting_P,
                               Boiling_P, density, HeatOfFusion,HeatOfEvop,ThermalCound,SpecficHeat ])
            a = np.array(properties)
            c = a.flatten()
            #print(c)
            new_df = new_df.append([c])
header_name = ['AN_A', 'AM-A', 'P_A', 'G_A', 'IE_main1_A', 'IE_main2_A', 'EAf_A', 'PEneg_A', 'AEneg_A','Vand_R_A', 'Cov_R_A', 'Atom_R_A', 'Melting_P',
                               'Boiling_P_A', 'density_A', 'HeatOfFusion_A','HeatOfEvop_A','ThermalCound_A','SpecficHeat_A']  
new_df = pd.read_csv("features_A.csv", header = None, skiprows=1, names= header_name) 
new_df.head()
# new_df.to_csv("features_A.csv")  
# pd.read_csv('features_A.csv').head()

AN_A       AM-A  P_A   G_A  IE_main1_A  IE_main2_A    EAf_A  PEneg_A  \
0  30.0  65.380000  4.0  12.0    9.394199    17.96439      NaN     1.65   
0  30.0  65.380000  4.0  12.0    9.394199    17.96439      NaN     1.65   
0  30.0  65.380000  4.0  12.0    9.394199    17.96439      NaN     1.65   
0  13.0  26.981538  3.0  13.0    5.985768    18.82855  0.43283     1.61   
0  13.0  26.981538  3.0  13.0    5.985768    18.82855  0.43283     1.61   

   AEneg_A  Vand_R_A  Cov_R_A  Atom_R_A  Melting_P  Boiling_P_A  density_A  \
0    9.395     201.0    118.0     138.0     692.73       1180.0     7.1330   
0    9.395     201.0    118.0     138.0     692.73       1180.0     7.1330   
0    9.395     201.0    118.0     138.0     692.73       1180.0     7.1330   
0    9.539     184.0    126.0     143.0     933.50       2740.0     2.6989   
0    9.539     184.0    126.0     143.0     933.50       2740.0     2.6989   

   HeatOfFusion_A  HeatOfEvop_A  ThermalCound_A  SpecficHeat_A  
0            7.28         114.8           116.0          0.388  
0            7.28         114.8           116.0          0.388  
0            7.28         114.8           116.0          0.388  
0           10.75         284.1           237.0          0.900  
0           10.75         284.1           237.0          0.900

### Edit  "features_A.csv" files

In [42]:
df_A=pd.read_csv("features_A1.csv")
df_A.shape

(99, 19)

In [43]:
df_A.head()

AN_A       AM-A  P_A  G_A  IE_main1_A  IE_main2_A    EAf_A  PEneg_A  \
0    30  65.380000    4   12    9.394199    17.96439      NaN     1.65   
1    30  65.380000    4   12    9.394199    17.96439      NaN     1.65   
2    30  65.380000    4   12    9.394199    17.96439      NaN     1.65   
3    13  26.981538    3   13    5.985768    18.82855  0.43283     1.61   
4    13  26.981538    3   13    5.985768    18.82855  0.43283     1.61   

   AEneg_A  Vand_R_A  Cov_R_A  Atom_R_A  Melting_P  Boiling_P_A  density_A  \
0    9.395       201      118       138     692.73         1180     7.1330   
1    9.395       201      118       138     692.73         1180     7.1330   
2    9.395       201      118       138     692.73         1180     7.1330   
3    9.539       184      126       143     933.50         2740     2.6989   
4    9.539       184      126       143     933.50         2740     2.6989   

   HeatOfFusion_A  HeatOfEvop_A  ThermalCound_A  SpecficHeat_A  
0            7.28         114.8           116.0          0.388  
1            7.28         114.8           116.0          0.388  
2            7.28         114.8           116.0          0.388  
3           10.75         284.1           237.0          0.900  
4           10.75         284.1           237.0          0.900

# For Element B

In [45]:
new_df1 = pd.DataFrame()
for item in df_U['B']:
    comp = mg.Composition(item)
    elements = comp.chemical_system.split("-")
    properties = []
    for i in elements:
            ele = element(i)  
            AN = ele.atomic_number
            AM = ele.mass
            P  = ele.period
            G = ele.group_id    
            IE = ele.ionenergies           
            IE_main1 = IE[1] 
            IE_main2 = IE[2]            
            EAf= ele.electron_affinity
            PEneg = ele.en_pauling
            AEneg = ele.en_allen
            Vand_R = ele.vdw_radius
            Cov_R = ele.covalent_radius
            Atom_R = ele.atomic_radius
            Melting_P = ele.melting_point
            Boiling_P = ele.boiling_point
            density = ele.density
            HeatOfFusion = ele.fusion_heat 
            HeatOfEvop = ele.evaporation_heat
            ThermalCound = ele.thermal_conductivity
            SpecficHeat = ele.specific_heat
            properties.append([AN, AM, P, G, IE_main1, IE_main2, EAf, PEneg, AEneg,Vand_R, Cov_R, Atom_R, Melting_P,
                               Boiling_P, density, HeatOfFusion,HeatOfEvop,ThermalCound,SpecficHeat ])
            a = np.array(properties)
            c = a.flatten()
#             print(c)
            new_df1 = new_df1.append([c])
new_df1.to_csv("features_B.csv")  
header_name = ['AN_B', 'AM-B', 'P_B', 'G_B', 'IE_main1_B', 'IE_main2_B', 'EAf_B', 'PEneg_B', 'AEneg_B','Vand_R_B', 'Cov_R_B', 'Atom_R_B', 'Melting_P_B',
                               'Boiling_P_B', 'density_B', 'HeatOfFusion_B','HeatOfEvop_B','ThermalCound_B','SpecficHeat_B']  
new_df2 = pd.read_csv("features_B.csv", header = None, skiprows=1, names= header_name) 
new_df2.to_csv("features_B.csv")  
new_df2.head()            

AN_B    AM-B  P_B   G_B  IE_main1_B  IE_main2_B     EAf_B  PEneg_B  \
0  32.0  72.630  4.0  14.0    7.899435    15.93461  1.232712     2.01   
0  32.0  72.630  4.0  14.0    7.899435    15.93461  1.232712     2.01   
0  32.0  72.630  4.0  14.0    7.899435    15.93461  1.232712     2.01   
0  31.0  69.723  4.0  13.0    5.999302    20.51514  0.430000     1.81   
0  31.0  69.723  4.0  13.0    5.999302    20.51514  0.430000     1.81   

   AEneg_B  Vand_R_B  Cov_R_B  Atom_R_B  Melting_P_B  Boiling_P_B  density_B  \
0    11.80     211.0    121.0     137.0      1210.60       3103.0      5.323   
0    11.80     211.0    121.0     137.0      1210.60       3103.0      5.323   
0    11.80     211.0    121.0     137.0      1210.60       3103.0      5.323   
0    10.39     187.0    124.0     141.0       302.93       2676.0      5.910   
0    10.39     187.0    124.0     141.0       302.93       2676.0      5.910   

   HeatOfFusion_B  HeatOfEvop_B  ThermalCound_B  SpecficHeat_B  
0           36.80         328.0            60.2          0.322  
0           36.80         328.0            60.2          0.322  
0           36.80         328.0            60.2          0.322  
0            5.59         270.3            28.1          0.372  
0            5.59         270.3            28.1          0.372

In [46]:
df_B=pd.read_csv("features_B1.csv")
df_B.head()

AN_B    AM-B  P_B  G_B  IE_main1_B  IE_main2_B     EAf_B  PEneg_B  AEneg_B  \
0    32  72.630    4   14    7.899435    15.93461  1.232712     2.01    11.80   
1    32  72.630    4   14    7.899435    15.93461  1.232712     2.01    11.80   
2    32  72.630    4   14    7.899435    15.93461  1.232712     2.01    11.80   
3    31  69.723    4   13    5.999302    20.51514  0.430000     1.81    10.39   
4    31  69.723    4   13    5.999302    20.51514  0.430000     1.81    10.39   

   Vand_R_B  Cov_R_B  Atom_R_B  Melting_P_B  Boiling_P_B  density_B  \
0       211      121       137      1210.60         3103      5.323   
1       211      121       137      1210.60         3103      5.323   
2       211      121       137      1210.60         3103      5.323   
3       187      124       141       302.93         2676      5.910   
4       187      124       141       302.93         2676      5.910   

   HeatOfFusion_B  HeatOfEvop_B  ThermalCound_B  SpecficHeat_B  
0           36.80         328.0            60.2          0.322  
1           36.80         328.0            60.2          0.322  
2           36.80         328.0            60.2          0.322  
3            5.59         270.3            28.1          0.372  
4            5.59         270.3            28.1          0.372

In [47]:
df_A.reset_index(drop=True, inplace=True)
df_B.reset_index(drop=True, inplace=True)
df_AB = pd.concat([df1,df3,df_A, df_B], axis=1)
df_AB[:50]

compound   A   B   X  AN_A        AM-A  P_A  G_A  IE_main1_A  IE_main2_A  \
0    ZnGeN2    Zn  Ge   N    30   65.380000    4   12    9.394199   17.964390   
1    ZnGeN2    Zn  Ge   N    30   65.380000    4   12    9.394199   17.964390   
2    ZnGeN2    Zn  Ge   N    30   65.380000    4   12    9.394199   17.964390   
3    AlGaN2    Al  Ga   N    13   26.981538    3   13    5.985768   18.828550   
4    AlGaN2    Al  Ga   N    13   26.981538    3   13    5.985768   18.828550   
5    AlGaN2    Al  Ga   N    13   26.981538    3   13    5.985768   18.828550   
6    ZnSiN2    Zn  Si   N    30   65.380000    4   12    9.394199   17.964390   
7    ZnSiN2    Zn  Si   N    30   65.380000    4   12    9.394199   17.964390   
8    ZnSiN2    Zn  Si   N    30   65.380000    4   12    9.394199   17.964390   
9   AlGaAs2    Al  Ga  As    13   26.981538    3   13    5.985768   18.828550   
10  AlGaAs2    Al  Ga  As    13   26.981538    3   13    5.985768   18.828550   
11  AlGaAs2    Al  Ga  As    13   26.981538    3   13    5.985768   18.828550   
12   AlGaP2    Al  Ga   P    13   26.981538    3   13    5.985768   18.828550   
13    AlGaP2   Al  Ga   P    13   26.981538    3   13    5.985768   18.828550   
14    AlGaP2   Al  Ga   P    13   26.981538    3   13    5.985768   18.828550   
15   GaInN2    Ga  In   N    31   69.723000    4   13    5.999302   20.515140   
16   GaInN2    Ga  In   N    31   69.723000    4   13    5.999302   20.515140   
17   GaInN2    Ga  In   N    31   69.723000    4   13    5.999302   20.515140   
18   AlInN2    Al  In   N    13   26.981538    3   13    5.985768   18.828550   
19   AlInN2    Al  In   N    13   26.981538    3   13    5.985768   18.828550   
20   AlInN2    Al  In   N    13   26.981538    3   13    5.985768   18.828550   
21   CdSiN2    Cd  Si   N    48  112.414000    5   12    8.993822   16.908312   
22   CdSiN2    Cd  Si   N    48  112.414000    5   12    8.993822   16.908312   
23   CdSiN2    Cd  Si   N    48  112.414000    5   12    8.993822   16.908312   
24   ZnGeP2    Zn  Ge   P    30   65.380000    4   12    9.394199   17.964390   
25   ZnGeP2    Zn  Ge   P    30   65.380000    4   12    9.394199   17.964390   
26   ZnGeP2    Zn  Ge   P    30   65.380000    4   12    9.394199   17.964390   
27   CdGeN2    Cd  Ge   N    48  112.414000    5   12    8.993822   16.908312   
28   CdGeN2    Cd  Ge   N    48  112.414000    5   12    8.993822   16.908312   
29   CdGeN2    Cd  Ge   N    48  112.414000    5   12    8.993822   16.908312   
30   GaInP2    Ga  In   P    31   69.723000    4   13    5.999302   20.515140   
31   GaInP2    Ga  In   P    31   69.723000    4   13    5.999302   20.515140   
32   GaInP2    Ga  In   P    31   69.723000    4   13    5.999302   20.515140   
33   AlInP2    Al  In   P    13   26.981538    3   13    5.985768   18.828550   
34   AlInP2    Al  In   P    13   26.981538    3   13    5.985768   18.828550   
35   AlInP2    Al  In   P    13   26.981538    3   13    5.985768   18.828550   
36  AlInAs2    Al  In  As    13   26.981538    3   13    5.985768   18.828550   
37   AlInAs2   Al  In  As    13   26.981538    3   13    5.985768   18.828550   
38   AlInAs2   Al  In  As    13   26.981538    3   13    5.985768   18.828550   
39  GaInAs2    Ga  In  As    31   69.723000    4   13    5.999302   20.515140   
40  GaInAs2    Ga  In  As    31   69.723000    4   13    5.999302   20.515140   
41  GaInAs2    Ga  In  As    31   69.723000    4   13    5.999302   20.515140   
42  CdSiAs2    Cd  Si  As    48  112.414000    5   12    8.993822   16.908312   
43  CdSiAs2    Cd  Si  As    48  112.414000    5   12    8.993822   16.908312   
44  CdSiAs2    Cd  Si  As    48  112.414000    5   12    8.993822   16.908312   
45  CdGeAs2    Cd  Ge  As    48  112.414000    5   12    8.993822   16.908312   
46  CdGeAs2    Cd  Ge  As    48  112.414000    5   12    8.993822   16.908312   
47  CdGeAs2    Cd  Ge  As    48  112.414000    5   12    8.993822   16.908312   
48  ZnGeAs2    Zn  Ge  As    30   65

In [49]:
df_AB.to_csv("features_AB.csv")  
pd.read_csv("features_AB.csv").head()

Unnamed: 0  compound   A   B  X  AN_A       AM-A  P_A  G_A  IE_main1_A  \
0           0  ZnGeN2    Zn  Ge  N    30  65.380000    4   12    9.394199   
1           1  ZnGeN2    Zn  Ge  N    30  65.380000    4   12    9.394199   
2           2  ZnGeN2    Zn  Ge  N    30  65.380000    4   12    9.394199   
3           3  AlGaN2    Al  Ga  N    13  26.981538    3   13    5.985768   
4           4  AlGaN2    Al  Ga  N    13  26.981538    3   13    5.985768   

   ...  Vand_R_B  Cov_R_B  Atom_R_B  Melting_P_B  Boiling_P_B  density_B  \
0  ...       211      121       137      1210.60         3103      5.323   
1  ...       211      121       137      1210.60         3103      5.323   
2  ...       211      121       137      1210.60         3103      5.323   
3  ...       187      124       141       302.93         2676      5.910   
4  ...       187      124       141       302.93         2676      5.910   

   HeatOfFusion_B  HeatOfEvop_B  ThermalCound_B  SpecficHeat_B  
0           36.80         328.0            60.2          0.322  
1           36.80         328.0            60.2          0.322  
2           36.80         328.0            60.2          0.322  
3            5.59         270.3            28.1          0.372  
4            5.59         270.3            28.1          0.372  

[5 rows x 43 columns]